In [3]:
from herramientas.librerias import *
%config InlineBackend.figure_format = 'svg' # Makes the images look nice
%matplotlib inline
## Se configura la forma de visualizar los vectores con el uso latex y markdown
latex_formatter = get_ipython().display_formatter.formatters["text/latex"]
latex_formatter.for_type(np.ndarray, lambda m: to_ltx(m, print_out=False))

# Construcción de Bloques de circuitos

In [4]:
def suma_bin(given_circuit,n): 

    given_circuit.cx(n,(n+1)) 
    given_circuit.cx(n,(n+2))
    return given_circuit


def U_c(): 
    
    sums=QuantumCircuit(4,name="U_c")
    sums.ccx(1,2,3) 
    sums.cx(1,2)
    sums.ccx(0,2,3)    
    return sums


def U_s(): 
    
    sums=QuantumCircuit(3,name="U_s")
    sums.cx(1,2)
    sums.cx(0,2) 
    return sums

In [5]:
def sumador_bin(num_bits):
    numb=num_bits
    n=numb*3+2
    b=QuantumCircuit(n)

    for i in range(0,(n-4),3):
        b.append(U_c(),b.qubits[i:(i+4)])  

    b.cx(n-4,n-3)
    ss=U_s()
    b.append(ss,b.qubits[(n-5):(n-2)])  

    for i in range((n-8),-3,-3):
        b.append(U_c(),b.qubits[i:(i+4)])
        b.append(U_s(),b.qubits[i:(i+3)]) 
    
    return b

h=sumador_bin(3)

In [7]:
h.draw()

┌──────┐                                             ┌──────┐┌──────┐
 q_0: ┤0     ├─────────────────────────────────────────────┤0     ├┤0     ├
      │      │                                             │      ││      │
 q_1: ┤1     ├─────────────────────────────────────────────┤1     ├┤1 U_s ├
      │  U_c │                                             │  U_c ││      │
 q_2: ┤2     ├─────────────────────────────────────────────┤2     ├┤2     ├
      │      │┌──────┐                     ┌──────┐┌──────┐│      │└──────┘
 q_3: ┤3     ├┤0     ├─────────────────────┤0     ├┤0     ├┤3     ├────────
      └──────┘│      │                     │      ││      │└──────┘        
 q_4: ────────┤1     ├─────────────────────┤1     ├┤1 U_s ├────────────────
              │  U_c │                     │  U_c ││      │                
 q_5: ────────┤2     ├─────────────────────┤2     ├┤2     ├────────────────
              │      │┌──────┐     ┌──────┐│      │└──────┘                
 q_6: ────────┤3     ├┤0     ├─────┤0     ├┤3     ├────────────────────────
              └──────┘│      │     │      │└──────┘                        
 q_7: ────────────────┤1     ├──■──┤1 U_s ├────────────────────────────────
                      │  U_c │┌─┴─┐│      │                                
 q_8: ────────────────┤2     ├┤ X ├┤2     ├────────────────────────────────
                      │      │└───┘└──────┘                                
 q_9: ────────────────┤3     ├─────────────────────────────────────────────
                      └──────┘                                             
q_10: ─────────────────────────────────────────────────────────────────────

# Creación de circuitos por medio de Append

In [8]:
a=QuantumCircuit(8)

for i in range(0,6,3):
    suma_bin(a,i) 

In [10]:
sums=QuantumCircuit(3)
sums.cx(0,1) 
sums.cx(0,2)

b=QuantumCircuit(8)

b.x(0)
b.append(sums,b.qubits[1:4])  

a.draw()

q_0: ──■────■──
     ┌─┴─┐  │  
q_1: ┤ X ├──┼──
     └───┘┌─┴─┐
q_2: ─────┤ X ├
          └───┘
q_3: ──■────■──
     ┌─┴─┐  │  
q_4: ┤ X ├──┼──
     └───┘┌─┴─┐
q_5: ─────┤ X ├
          └───┘
q_6: ──────────
               
q_7: ──────────

# Creación de multiples entradas para evolucionar en el circuito

In [11]:
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import state_fidelity

In [12]:
sumador=a

for i in range(10):
    print("{0:b}".format(i))
    entrada = Statevector.from_int(i,dims=(2,2,2,2,2,2,2,2))
    evolucion = entrada.evolve(sumador)
    counts = evolucion.sample_counts(shots=1000)
    print(counts)

0
{'00000000': 1000}
1
{'00000111': 1000}
10
{'00000010': 1000}
11
{'00000101': 1000}
100
{'00000100': 1000}
101
{'00000011': 1000}
110
{'00000110': 1000}
111
{'00000001': 1000}
1000
{'00111000': 1000}
1001
{'00111111': 1000}


# Organizar una entrada con cadenas de bits

In [13]:
# get bit representation of a number
zero=0
#weight_binary = '{0:b}'.format(int(weight)).rjust(self.num_sum_qubits, '0')[::-1]
zero_binary = '{0:b}'.format(int(zero)).rjust(10, '0')[::-1]

Numa=3
Numb=4
Num_binary_a = '{0:b}'.format(int(Numa))
Num_binary_b = '{0:b}'.format(int(Numb))

In [15]:
a='0'
for j, bit_a in enumerate(Num_binary_a):
    if bit_a == '1':
        a="1"+a
        bit_b = Num_binary_b[j]
        if bit_b == '1':
            a="1"+a
        else:
            a="0"+a
    else:
        a="0"+a
        bit_b = Num_binary_b[j]
        if bit_b == '1':
            a="1"+a
        else:
            a="0"+a
    a="0"+a

a

'0010110'